In [1]:
import pandas as pd
import numpy as np
import sklearn
import random
import string
import os
import re
import pprint
import queue
import requests
import requests.auth

In [2]:
def get_reddit(query):
    client_auth = requests.auth.HTTPBasicAuth('BGPeMGr5pYZxQA', '0tsNJmqQr7LnN_y9AEuTn0XXIDs')
    post_data = {"grant_type": "password", "username": "dataorelse", "password": "password"}
    headers = {"User-Agent": "ChangeMeClient/0.1 by dataorelse"}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    auth_resp = response.json()
    headers = {
        "Authorization": "bearer %s" % auth_resp["access_token"], 
        "User-Agent": "ChangeMeClient/0.1 by YourUsername",
    }
    response = requests.get("https://oauth.reddit.com" + query, headers=headers)
    r  = response.json()
    #pprint.pprint(r)
    return(r)

In [3]:
r = get_reddit("/r/politics")

In [4]:
after = r["data"]["after"]
threads = [r["data"]["children"][idx]["data"] for idx in range(r["data"]["children"].__len__())]
front_page = [{k:thread[k] for k in ['author', 'permalink', 'name', 'domain', 'url', 'id', 'num_comments', 'locked', 'likes', 'score', 'downs', 'title', 'subreddit', 'subreddit_id', 
                                 'gilded', 
                                 'ups', 'edited', 'hidden', 'saved', 'stickied', 'over_18'] }
         for thread in threads]

In [5]:
top_thread = max([(x["num_comments"], x["permalink"]) for x in front_page])[1]

In [6]:
top_thread

'/r/politics/comments/5qazu1/the_state_departments_entire_senior_management/'

In [7]:
s = get_reddit('/r/politics/comments/5qazu1/the_state_departments_entire_senior_management/t1_dcy3cqp')

In [8]:
# first element of returned array is the post info
pprint.pprint(s[0])

{'data': {'after': None,
          'before': None,
          'children': [{'data': {'approved_by': None,
                                 'archived': False,
                                 'author': 'The_Iceman2288',
                                 'author_flair_css_class': None,
                                 'author_flair_text': None,
                                 'banned_by': None,
                                 'clicked': False,
                                 'contest_mode': False,
                                 'created': 1485475763.0,
                                 'created_utc': 1485446963.0,
                                 'distinguished': None,
                                 'domain': 'washingtonpost.com',
                                 'downs': 0,
                                 'edited': False,
                                 'gilded': 0,
                                 'hidden': False,
                                 'hide_score': False,
            

In [40]:
#got a thread
#the next (and only other) element of the returned array contains the comments

comment_root = s[1]
stack = []

stack = populate_dfs_stack(stack=stack, node=comment_root)
while stack!=[]:
    n = stack.pop()
    stack = populate_dfs_stack(stack=stack, node=n)

#data = comment_root["data"] if comment_root.__class__() == {} and comment_root.keys().__contains__("data") else None
#children = data["children"]  if data.__class__() == {} and data.keys().__contains__("children") else None
#assert children is not None and children.__class__() == []
#if children:
#    stack = populate_dfs_stack(stack, children.__iter__())

#comment_data = comment["data"] if comment.keys().__contains__("data") else None
#comment_body = comment_data["body"] if comment_data.keys().__contains__("body") else None
#if comment_body is not None:
#    result.append(comment_body)

[{'data': {'approved_by': None,
   'archived': False,
   'author': 'AutoModerator',
   'author_flair_css_class': None,
   'author_flair_text': None,
   'banned_by': None,
   'body': "\nAs a reminder, this subreddit [is for civil discussion.](https://www.reddit.com/r/politics/wiki/rulesandregs#wiki_please_be_civil)\n\n* Do not call other users trolls, morons, children, or anything else clever you may think of. [Personal attacks, whether explicit or implicit, are not permitted.](https://www.reddit.com/r/politics/wiki/rulesandregs#wiki_no_personal_attacks)\n\n* Do not accuse other users of being shills. If you believe that a user is a shill, the proper conduct is to report the user or send us a modmail.\n\n* In general, don't be a jerk. Don't bait people, don't use hate speech, etc. Attack ideas, not users.\n\n* Do not downvote comments because you disagree with them, and be willing to upvote quality comments whether you agree with the opinions held or not.\n\nIncivility will result in a 

In [45]:
def populate_dfs_stack(stack, node):
    if node.__class__() == {}  and node.keys().__contains__("data"):
        data = node["data"]
        if data.__class__() == {} and data.keys().__contains__("children"):
            children = data["children"]
            assert children.__class__() == []
            iterable = children.__iter__()
            while iterable.__length_hint__() > 0:
                n = iterable.__next__()
                stack.append(n)
    return stack

def extract_comment_body(comment):
    if comment.keys().__contains__("body"):
        if comment.keys().__contains__("dialog_trail"):
            comment["dialog_trail"].append(comment["body"])
        else:
            comment["dialog_trail"] = []
            comment["dialog_trail"].append(comment["body"])
    return comment
        

In [46]:
c = stack[1]["data"]

In [47]:
extract_comment_body(c)

{'approved_by': None,
 'archived': False,
 'author': 'Gravel090',
 'author_flair_css_class': None,
 'author_flair_text': None,
 'banned_by': None,
 'body': "You have to wonder if the exodus was because they got forced out or because they heard what the boss man was saying and realized they didn't want to be implicated in the terrible shit to come.",
 'body_html': '&lt;div class="md"&gt;&lt;p&gt;You have to wonder if the exodus was because they got forced out or because they heard what the boss man was saying and realized they didn&amp;#39;t want to be implicated in the terrible shit to come.&lt;/p&gt;\n&lt;/div&gt;',
 'controversiality': 0,
 'created': 1485476153.0,
 'created_utc': 1485447353.0,
 'dialog_trail': ["You have to wonder if the exodus was because they got forced out or because they heard what the boss man was saying and realized they didn't want to be implicated in the terrible shit to come."],
 'distinguished': None,
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'id': 'dcx